In [ ]:
# !pip install selenium

In [ ]:
# !pip install webdriver_manager

# 자동로그인을 하기 위한 사용자 정보 수집 코드 chrome_profile

In [5]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# --- Selenium 옵션 설정 ---
options = Options()
options.add_experimental_option("detach", True)

# ★★★ 사용자 프로필 경로 설정 ★★★
profile_path = os.path.join(os.getcwd(), "chrome_profile")
options.add_argument(f"--user-data-dir={profile_path}")

# --- 자동화 탐지 회피 설정 ---
options.add_argument("start-maximized")
# ... (이전과 동일한 나머지 옵션들) ...
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Whale/4.32.315.15 Safari/537.36")
options.add_argument("referrer-policy=unsafe-url")
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("lang=ko_KR")

# 드라이버 실행
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

# 네이버 메인으로 이동
driver.get("https://www.naver.com")

# ★★★★★ 가장 중요한 부분 ★★★★★
input("""
브라우저가 열렸습니다. 다음 단계를 꼭 순서대로 진행해주세요:
1. ID와 비밀번호를 입력합니다.
2. [로그인 상태 유지] 체크박스를 ★반드시★ 선택합니다.
3. [로그인] 버튼을 클릭합니다.
4. 로그인이 완료된 것을 확인한 후, 이 터미널로 돌아와 Enter 키를 누르세요...
""")

print("로그인 프로필이 저장되었습니다. 브라우저를 닫으셔도 됩니다.")
print("이제부터는 '세션 재사용' 코드를 실행해주세요.")

driver.quit()



브라우저가 열렸습니다. 다음 단계를 꼭 순서대로 진행해주세요:
1. ID와 비밀번호를 입력합니다.
2. [로그인 상태 유지] 체크박스를 ★반드시★ 선택합니다.
3. [로그인] 버튼을 클릭합니다.
4. 로그인이 완료된 것을 확인한 후, 이 터미널로 돌아와 Enter 키를 누르세요...

로그인 프로필이 저장되었습니다. 브라우저를 닫으셔도 됩니다.
이제부터는 '세션 재사용' 코드를 실행해주세요.


In [9]:
import os
import time
import csv
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException


def setup_driver():
    options = Options()
    profile_path = os.path.join(os.getcwd(), "chrome_profile")
    options.add_argument(f"--user-data-dir={profile_path}")
    options.add_experimental_option("detach", True)
    options.add_argument("start-maximized")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Whale/4.32.315.15 Safari/537.36")
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument("lang=ko_KR")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


def parse_cafe_date(date_str, collected_date):
    try:
        if ":" in date_str:
            return collected_date
        else:
            return datetime.strptime(date_str.strip(), '%Y.%m.%d.').date()
    except ValueError:
        return collected_date


def safe_wait(driver, condition, timeout=20, retries=3, delay=2):
    for attempt in range(retries):
        try:
            wait = WebDriverWait(driver, timeout)
            return wait.until(condition)
        except TimeoutException:
            print(f"⏳ 요소 로딩 지연 (시도 {attempt+1}/{retries})", end="\r")
            time.sleep(delay)
    raise TimeoutException(f"⚠️ 요소 로딩 실패: {condition}", end="\r")


def handle_login(driver, wait):
    try:
        login_btn = driver.find_element(By.ID, "gnb_login_button")
        if login_btn.is_displayed() and login_btn.is_enabled():
            print("🔐 로그인 버튼 발견 → 로그인 시도")
            login_btn.click()
            time.sleep(1)
            pw_box = wait.until(EC.presence_of_element_located((By.ID, "pw")))
            pw_box.send_keys(Keys.ENTER)
            time.sleep(5)
        else:
            print("✅ 로그인 버튼 비활성화 상태 → 로그인 유지 중", end="\r")
    except NoSuchElementException:
        print("✅ 로그인 버튼 없음 → 로그인 유지 중", end="\r")
    except Exception as e:
        print(f"⚠️ 로그인 처리 중 예외 발생: {e}")


print("네이버카페 게시판 크롤링 시작")
driver = setup_driver()
wait = WebDriverWait(driver, 20)

base_url = "https://cafe.naver.com/f-e/cafes/19997929/menus/0?q=%EC%8B%9C%ED%97%98&ta=SUBJECT&page=1&size=50"
csv_filename = 'crawling_results_1year_inline.csv'

with open(csv_filename, 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.writer(f)
    writer.writerow(['작성일', '제목', '본문', '댓글', 'URL'])

    one_year_ago = (datetime.now() - timedelta(days=365)).date()
    page_count = 1
    stop_collecting = False

    while not stop_collecting:
        target_url = f"{base_url}&page={page_count}"

        # google.com 튕김 방지
        current_url = driver.current_url
        if "google.com" in current_url:
            print("🚨 google.com으로 튕김 감지 → 복구 시도")
            driver.get(target_url)
            time.sleep(3)
            handle_login(driver, wait)

        print(f"\n📄 페이지 {page_count} 진입")
        driver.get(target_url)
        time.sleep(3)
        handle_login(driver, wait)

        try:
            tbodys = safe_wait(driver, EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table.article-table tbody")))
            article_rows = tbodys[-1].find_elements(By.TAG_NAME, "tr")
        except:
            print("❌ 게시물 목록 로딩 실패")
            break

        for i in range(len(article_rows)):
            print(f"  └▷ {i+1}번째 게시물 처리 중...                      ", end="\r")

            # 튕김 여부 체크 후 복구
            if "google.com" in driver.current_url:
                print("🚨 게시물 처리 중 튕김 → 페이지 복구", end="\r")
                driver.get(target_url)
                time.sleep(3)
                handle_login(driver, wait)
                break

            try:
                tbodys = safe_wait(driver, EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table.article-table tbody")))
                article_rows = tbodys[-1].find_elements(By.TAG_NAME, "tr")
                row = article_rows[i]

                date_element = row.find_element(By.CSS_SELECTOR, "td.td_normal.type_date")
                date_str = date_element.text.strip()
                collected_date = datetime.now().date()
                article_date = parse_cafe_date(date_str, collected_date)

                if article_date < one_year_ago:
                    print("✅ 1년 이전 게시물이니 수집 종료")
                    stop_collecting = True
                    break

                link_element = row.find_element(By.CSS_SELECTOR, "a.article")
                link_element.click()

                safe_wait(driver, EC.frame_to_be_available_and_switch_to_it((By.ID, "cafe_main")))

                try:
                    while True:
                        driver.execute_script("window.scrollBy(0, 500);")
                        time.sleep(0.5)
                        new_height = driver.execute_script("return window.scrollY + window.innerHeight;")
                        total_height = driver.execute_script("return document.body.scrollHeight;")
                        if new_height >= total_height:
                            break
                except Exception as e:
                    print(f"❌ 반복 스크롤 실패: {e}")

                title = safe_wait(driver, EC.presence_of_element_located((By.CSS_SELECTOR, ".title_text"))).text
                content = safe_wait(driver, EC.presence_of_element_located((By.CSS_SELECTOR, ".se-main-container, .content.CafeViewer"))).text

                reply_list = []
                try:
                    replies = driver.find_elements(By.CSS_SELECTOR, "div.CommentBox > div:nth-child(2) > ul > li")
                    for reply in replies:
                        try:
                            reply_text = reply.find_element(By.CSS_SELECTOR, "div.comment_text_box > p > span").text
                            reply_list.append(reply_text)
                        except:
                            reply_list.append("(삭제된 댓글 또는 비정상 구조)")
                except:
                    pass

                writer.writerow([
                    article_date.strftime('%Y.%m.%d'),
                    ' '.join(title.split()),
                    ' '.join(content.split()),
                    ' | '.join(reply_list),
                    driver.current_url
                ])
                print(f"  {i+1}번째 게시물 ✅ 저장 완료: {title[:30]}...", end="\r")
                time.sleep(5)

            except Exception as e:
                print(f"    ❌ 오류 발생: {e}")
            finally:
                try:
                    driver.switch_to.default_content()
                    driver.back()
                    time.sleep(3)
                except:
                    pass

        if stop_collecting:
            break

        try:
            pagination = driver.find_element(By.CSS_SELECTOR, ".Pagination")
            next_button = pagination.find_element(By.XPATH, ".//button[@aria-pressed='true']/following-sibling::button[1]")
            if "disabled" in next_button.get_attribute("class"):
                print("📉 마지막 페이지입니다.")
                break
            page_count += 1
        except NoSuchElementException:
            print("📉 다음 페이지 없음 → 종료")
            break


print(f"\n✅ 모든 게시물 수집 완료 → '{csv_filename}'에 저장됨")
driver.quit()


네이버카페 게시판 크롤링 시작

📄 페이지 1 진입
    ❌ 오류 발생: Message: no such element: Unable to locate element: {"method":"css selector","selector":"td.td_normal.type_date"}
  (Session info: chrome=137.0.7151.69); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x1213b03+62899]
	GetHandleVerifier [0x0x1213b44+62964]
	(No symbol) [0x0x10410f3]
	(No symbol) [0x0x108980e]
	(No symbol) [0x0x1089bab]
	(No symbol) [0x0x107ef51]
	(No symbol) [0x0x10ae554]
	(No symbol) [0x0x107ee74]
	(No symbol) [0x0x10ae784]
	(No symbol) [0x0x10cfd81]
	(No symbol) [0x0x10ae306]
	(No symbol) [0x0x107d670]
	(No symbol) [0x0x107e4e4]
	GetHandleVerifier [0x0x1474793+2556483]
	GetHandleVerifier [0x0x146fd02+2537394]
	GetHandleVerifier [0x0x123a2fa+220586]
	GetHandleVerifier [0x0x122aae8+157080]
	GetHandleVerifier [0x0x123141d+184013]
	GetHandleVerifier [0x0x121ba68+95512]
	GetHandleVerifier [0x0x121bc10

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=137.0.7151.69)
Stacktrace:
	GetHandleVerifier [0x0x1213b03+62899]
	GetHandleVerifier [0x0x1213b44+62964]
	(No symbol) [0x0x10410f3]
	(No symbol) [0x0x101ff59]
	(No symbol) [0x0x10b4f7e]
	(No symbol) [0x0x10cf6a9]
	(No symbol) [0x0x10ae306]
	(No symbol) [0x0x107d670]
	(No symbol) [0x0x107e4e4]
	GetHandleVerifier [0x0x1474793+2556483]
	GetHandleVerifier [0x0x146fd02+2537394]
	GetHandleVerifier [0x0x123a2fa+220586]
	GetHandleVerifier [0x0x122aae8+157080]
	GetHandleVerifier [0x0x123141d+184013]
	GetHandleVerifier [0x0x121ba68+95512]
	GetHandleVerifier [0x0x121bc10+95936]
	GetHandleVerifier [0x0x1206b5a+9738]
	BaseThreadInitThunk [0x0x764e5d49+25]
	RtlInitializeExceptionChain [0x0x77dbd09b+107]
	RtlGetAppContainerNamedObjectPath [0x0x77dbd021+561]
